In [32]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [24]:

print(os.getenv("PINECONE_API_KEY"))  

None


In [28]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",dimension=1536, metric="cosine",spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [30]:
import json
data=json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Chen',
  'subject': 'Calculus II',
  'stars': 5,
  'review': 'Dr. Chen is an exceptional professor. She is extremely knowledgeable and passionate about the subject, and her explanations are clear and concise. The coursework is challenging but manageable, and she provides ample support during office hours. I highly recommend taking any course with Dr. Chen.'},
 {'professor': 'Professor John Smith',
  'subject': 'Principles of Microeconomics',
  'stars': 3,
  'review': 'Professor Smith is knowledgeable about the subject matter, but his lectures can be dry and difficult to follow at times. The grading seems a bit subjective, and I wish he provided more feedback on assignments.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Organic Chemistry I',
  'stars': 4,
  'review': 'Dr. Lee is passionate about the subject and really tries to help students understand the material. The lab component is well-organized, and she is always willing to answer questions during offi

In [36]:
processed_data=[]
client= OpenAI()

for review in data['reviews']:
    response =client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding= response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [38]:
processed_data[0]

{'values': [0.0007057121,
  -0.022521522,
  0.010078319,
  0.049135637,
  0.033083845,
  0.023575302,
  -0.0036637294,
  0.030878255,
  -0.038622327,
  0.01818386,
  0.0039792513,
  0.010488804,
  -0.013307057,
  -0.008840739,
  0.0057467865,
  0.05558086,
  -0.015806725,
  0.0011096873,
  0.019592987,
  0.033524964,
  0.033377923,
  -0.0068740877,
  0.026222011,
  -0.013245791,
  -0.06861834,
  -0.03724996,
  0.025033442,
  0.0021718931,
  0.0069904937,
  -0.002034044,
  0.037421506,
  -0.016125312,
  -0.015108289,
  -0.025119215,
  -0.0733236,
  0.06577559,
  -0.016162071,
  0.05749237,
  0.017828517,
  -0.017473172,
  0.029824473,
  0.043254063,
  -0.030167565,
  0.020070866,
  -0.016848255,
  -0.0141280275,
  -0.015684193,
  0.028182535,
  0.016664455,
  0.016321363,
  -0.06317789,
  0.036759827,
  0.042371828,
  -0.005961219,
  -0.07714663,
  0.05607099,
  0.018526953,
  0.05131672,
  0.0008025896,
  -0.029505888,
  0.07449992,
  -0.073911764,
  -0.018980324,
  -0.008754966,
  -0.

In [39]:
index=pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 21}

In [40]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}